# 시간대별 지하철 혼잡도 종합 연관성 분석

이 노트북은 지하철 혼잡도와 다양한 도시 데이터 간의 종합적인 연관성을 분석합니다.

## 분석 대상
- **2호선, 4호선, 5호선** 역으로 한정

## 분석 데이터
1. **역세권 건물** - 용도별 건물 수, 연면적, 세대수
2. **추정매출** - 업종별, 시간대별 매출
3. **유동인구** - 시간대별, 요일별 유동인구
4. **직장인구** - 연령대별, 성별 직장인구
5. **생활인구** - 시간대별 생활인구

## 분석 목표
- 혼잡도에 영향을 미치는 주요 요인 파악
- 시간대별 혼잡도 패턴과 도시 특성 관계 규명
- 다중 회귀 분석을 통한 혼잡도 예측 모델 구축

In [1]:
from huggingface_hub import hf_hub_download
import sqlite3
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

repo_id = "alrq/subway"       # 데이터셋 리포지토리 ID
filename = "db/subway.db"     # 리포지토리 내 파일 경로
local_dir = "."               # 다운로드 받을 로컬 기본 경로 (이 경우 ./db/subway.db 로 저장됨)
# 파일 다운로드
# local_dir을 지정하면 리포지토리의 폴더 구조를 유지하며 파일을 저장합니다.
DB_PATH = hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    repo_type="dataset",
    local_dir=local_dir
)

DB_PATH = "../db/subway.db"

TARGET_LINES = ['2호선', '4호선', '5호선']

def get_connection():
    return sqlite3.connect(DB_PATH)

print("라이브러리 로드 완료")

c:\USERS\ADMINISTRATOR\DOCUMENTS\SUBWAY-CONGESTION-ANALYSIS\.VENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


라이브러리 로드 완료


## 1. 데이터 로드

In [2]:
conn = get_connection()

# 역 정보 로드 (2, 4, 5호선)
df_stations = pd.read_sql("""
SELECT s.station_id, s.station_name_kr, sr.station_code, 
       sr.admin_dong_code, sr.admin_dong_name, l.line_name
FROM Stations s
JOIN Station_Routes sr ON s.station_id = sr.station_id
JOIN Lines l ON sr.line_id = l.line_id
WHERE l.line_name IN ('2호선', '4호선', '5호선')
""", conn)

target_station_ids = df_stations['station_id'].unique().tolist()
target_station_codes = df_stations['station_code'].unique().tolist()
target_dong_codes = list(set([str(d)[:8] for d in df_stations['admin_dong_code'].dropna().unique()]))

print(f"분석 대상 호선: {TARGET_LINES}")
print(f"역 수: {len(target_station_ids)}개")
print(f"행정동 수: {len(target_dong_codes)}개")
print("\n호선별 역 수:")
print(df_stations.groupby('line_name')['station_id'].nunique())

분석 대상 호선: ['2호선', '4호선', '5호선']
역 수: 125개
행정동 수: 101개

호선별 역 수:
line_name
2호선    50
4호선    26
5호선    56
Name: station_id, dtype: int64


In [3]:
# 혼잡도 데이터
df_congestion = pd.read_sql(f"""
SELECT * FROM Station_Congestion
WHERE station_code IN ({','.join([f"'{c}'" for c in target_station_codes])})
""", conn)

# 시간대 분류
def categorize_time(slot):
    hour = ((5 * 60 + 30 + slot * 30) // 60) % 24
    if hour < 6: return '00_06'
    elif hour < 11: return '06_11'
    elif hour < 14: return '11_14'
    elif hour < 17: return '14_17'
    elif hour < 21: return '17_21'
    else: return '21_24'

df_congestion['time_period'] = df_congestion['time_slot'].apply(categorize_time)
df_congestion = df_congestion.merge(
    df_stations[['station_id', 'station_code']].drop_duplicates(), 
    on='station_code', how='left'
)

print(f"혼잡도 데이터: {len(df_congestion):,} rows")

혼잡도 데이터: 130,960 rows


In [4]:
# 역세권 건물 데이터
df_buildings = pd.read_sql(f"""
SELECT b.*, s.station_name_kr
FROM Station_Catchment_Buildings b
JOIN Stations s ON b.station_id = s.station_id
WHERE b.station_id IN ({','.join(map(str, target_station_ids))})
""", conn)

print(f"역세권 건물: {len(df_buildings):,} rows")

역세권 건물: 239,295 rows


In [5]:
# 추정매출 데이터
df_revenue_all = pd.read_sql("SELECT * FROM Dong_Estimated_Revenue", conn)
df_revenue_all['dong_short'] = df_revenue_all['admin_dong_code'].astype(str).str[:8]
df_revenue = df_revenue_all[df_revenue_all['dong_short'].isin(target_dong_codes)]

print(f"추정매출: {len(df_revenue):,} rows")

추정매출: 45,377 rows


In [6]:
# 유동인구 데이터
df_floating_all = pd.read_sql("SELECT * FROM Dong_Floating_Population", conn)
df_floating_all['dong_short'] = df_floating_all['admin_dong_code'].astype(str).str[:8]
df_floating = df_floating_all[df_floating_all['dong_short'].isin(target_dong_codes)]

# 직장인구 데이터
df_workplace_all = pd.read_sql("SELECT * FROM Dong_Workplace_Population", conn)
df_workplace_all['dong_short'] = df_workplace_all['admin_dong_code'].astype(str).str[:8]
df_workplace = df_workplace_all[df_workplace_all['dong_short'].isin(target_dong_codes)]

print(f"유동인구: {len(df_floating):,} rows")
print(f"직장인구: {len(df_workplace):,} rows")

유동인구: 1,034 rows
직장인구: 1,023 rows


## 2. 데이터 전처리 및 집계

In [7]:
# 역별 시간대별 혼잡도 집계 (평일)
df_cong_weekday = df_congestion[df_congestion['day_of_week'] == 0]

cong_pivot = df_cong_weekday.groupby(['station_id', 'time_period'])['congestion_level'].mean().unstack()
cong_pivot.columns = [f'cong_{c}' for c in cong_pivot.columns]
cong_pivot['cong_avg'] = cong_pivot.mean(axis=1)
cong_pivot['cong_peak'] = cong_pivot[['cong_06_11', 'cong_17_21']].max(axis=1)  # 출퇴근 피크
cong_pivot['cong_offpeak'] = cong_pivot[['cong_11_14', 'cong_14_17']].mean(axis=1)  # 비피크
cong_pivot = cong_pivot.reset_index()

print(f"역별 혼잡도: {len(cong_pivot)} 역")
display(cong_pivot.head())

역별 혼잡도: 125 역


,station_id,cong_00_06,cong_06_11,cong_11_14,cong_14_17,cong_17_21,cong_21_24,cong_avg,cong_peak,cong_offpeak
0,1,10.696667,29.017,30.335000,34.816667,38.18125,23.130000,27.696097,38.18125,32.575833
1,2,11.956667,39.710,37.485000,43.685000,57.98250,40.073333,38.482083,57.98250,40.585000
2,4,9.046667,36.010,26.070000,34.060000,46.35375,25.978333,29.586458,46.35375,30.065000
3,6,21.623333,54.663,44.910000,55.970000,64.88750,42.155000,47.368139,64.88750,50.440000
4,7,1.816667,11.767,5.906667,6.178333,5.65750,1.986667,5.552139,11.76700,6.042500


In [8]:
# 건물 용도 분류
def categorize_usage(u):
    if pd.isna(u): return '기타'
    u = str(u)
    if '주택' in u: return '주거'
    elif '근린생활' in u or '판매' in u: return '상업'
    elif '업무' in u: return '업무'
    elif '교육' in u: return '교육'
    elif '숙박' in u: return '숙박'
    elif '공장' in u or '창고' in u: return '산업'
    else: return '기타'

df_buildings['usage_cat'] = df_buildings['usage_type'].apply(categorize_usage)

# 역별 건물 특성 집계
building_stats = df_buildings.groupby('station_id').agg({
    'id': 'count',
    'height': 'mean',
    'floor_area': 'sum',
    'households': 'sum'
}).rename(columns={
    'id': 'bldg_count', 'height': 'bldg_avg_height',
    'floor_area': 'bldg_total_area', 'households': 'bldg_households'
})

# 용도별 건물 수
usage_pivot = df_buildings.groupby(['station_id', 'usage_cat']).size().unstack(fill_value=0)
usage_pivot.columns = [f'bldg_{c}' for c in usage_pivot.columns]

building_stats = building_stats.join(usage_pivot).reset_index()

# 용도 비율 계산
for col in ['bldg_주거', 'bldg_상업', 'bldg_업무']:
    if col in building_stats.columns:
        building_stats[f'{col}_ratio'] = building_stats[col] / building_stats['bldg_count']

print(f"역별 건물 특성: {len(building_stats)} 역")

역별 건물 특성: 119 역


In [9]:
# 역-행정동 매핑
station_dong = df_stations[['station_id', 'station_name_kr', 'admin_dong_code']].drop_duplicates(subset='station_id')
station_dong['dong_short'] = station_dong['admin_dong_code'].astype(str).str[:8]

# 최신 분기 매출 집계
latest_q = df_revenue['quarter_code'].max()
df_rev_latest = df_revenue[df_revenue['quarter_code'] == latest_q]

dong_revenue = df_rev_latest.groupby('dong_short').agg({
    'month_sales_amt': 'sum',
    'time_00_06_sales_amt': 'sum', 'time_06_11_sales_amt': 'sum',
    'time_11_14_sales_amt': 'sum', 'time_14_17_sales_amt': 'sum',
    'time_17_21_sales_amt': 'sum', 'time_21_24_sales_amt': 'sum',
    'weekday_sales_amt': 'sum', 'weekend_sales_amt': 'sum'
}).reset_index()

dong_revenue.columns = ['dong_short', 'sales_total', 'sales_00_06', 'sales_06_11',
                        'sales_11_14', 'sales_14_17', 'sales_17_21', 'sales_21_24',
                        'sales_weekday', 'sales_weekend']

# 매출 정규화 (억 단위)
for col in dong_revenue.columns[1:]:
    dong_revenue[col] = dong_revenue[col] / 1e8

print(f"행정동별 매출: {len(dong_revenue)} 행정동 (분기: {latest_q})")

행정동별 매출: 94 행정동 (분기: 20253)


In [10]:
# 유동인구 집계 (최신 분기)
latest_q_float = df_floating['quarter_code'].max()
df_float_latest = df_floating[df_floating['quarter_code'] == latest_q_float]

dong_floating = df_float_latest.groupby('dong_short').agg({
    'total_floating_pop': 'mean',
    'time_00_06_floating_pop': 'mean', 'time_06_11_floating_pop': 'mean',
    'time_11_14_floating_pop': 'mean', 'time_14_17_floating_pop': 'mean',
    'time_17_21_floating_pop': 'mean', 'time_21_24_floating_pop': 'mean'
}).reset_index()

dong_floating.columns = ['dong_short', 'float_total', 'float_00_06', 'float_06_11',
                         'float_11_14', 'float_14_17', 'float_17_21', 'float_21_24']

# 만 단위로 변환
for col in dong_floating.columns[1:]:
    dong_floating[col] = dong_floating[col] / 10000

print(f"행정동별 유동인구: {len(dong_floating)} 행정동")

행정동별 유동인구: 94 행정동


In [11]:
# 직장인구 집계
latest_q_work = df_workplace['quarter_code'].max()
df_work_latest = df_workplace[df_workplace['quarter_code'] == latest_q_work]

dong_workplace = df_work_latest.groupby('dong_short').agg({
    'total_pop': 'sum',
    'male_pop': 'sum', 'female_pop': 'sum',
    'age_20_pop': 'sum', 'age_30_pop': 'sum', 'age_40_pop': 'sum', 'age_50_pop': 'sum'
}).reset_index()

dong_workplace.columns = ['dong_short', 'work_total', 'work_male', 'work_female',
                          'work_age20', 'work_age30', 'work_age40', 'work_age50']

# 천 단위로 변환
for col in dong_workplace.columns[1:]:
    dong_workplace[col] = dong_workplace[col] / 1000

print(f"행정동별 직장인구: {len(dong_workplace)} 행정동")

행정동별 직장인구: 93 행정동


## 3. 데이터 통합

In [12]:
# 모든 데이터 통합
df_merged = station_dong[['station_id', 'station_name_kr', 'dong_short']].copy()

# 혼잡도
df_merged = df_merged.merge(cong_pivot, on='station_id', how='left')

# 건물
df_merged = df_merged.merge(building_stats, on='station_id', how='left')

# 매출
df_merged = df_merged.merge(dong_revenue, on='dong_short', how='left')

# 유동인구
df_merged = df_merged.merge(dong_floating, on='dong_short', how='left')

# 직장인구
df_merged = df_merged.merge(dong_workplace, on='dong_short', how='left')

# 결측치 제거
df_analysis = df_merged.dropna(subset=['cong_avg', 'bldg_count']).copy()

print(f"통합 데이터: {len(df_analysis)} 역")
print(f"\n컬럼 수: {len(df_analysis.columns)}")
print(f"컬럼 목록: {df_analysis.columns.tolist()}")

통합 데이터: 119 역

컬럼 수: 49
컬럼 목록: ['station_id', 'station_name_kr', 'dong_short', 'cong_00_06', 'cong_06_11', 'cong_11_14', 'cong_14_17', 'cong_17_21', 'cong_21_24', 'cong_avg', 'cong_peak', 'cong_offpeak', 'bldg_count', 'bldg_avg_height', 'bldg_total_area', 'bldg_households', 'bldg_교육', 'bldg_기타', 'bldg_산업', 'bldg_상업', 'bldg_숙박', 'bldg_업무', 'bldg_주거', 'bldg_주거_ratio', 'bldg_상업_ratio', 'bldg_업무_ratio', 'sales_total', 'sales_00_06', 'sales_06_11', 'sales_11_14', 'sales_14_17', 'sales_17_21', 'sales_21_24', 'sales_weekday', 'sales_weekend', 'float_total', 'float_00_06', 'float_06_11', 'float_11_14', 'float_14_17', 'float_17_21', 'float_21_24', 'work_total', 'work_male', 'work_female', 'work_age20', 'work_age30', 'work_age40', 'work_age50']


In [13]:
# 데이터 요약
print("=== 통합 데이터 요약 ===")
summary_cols = ['cong_avg', 'bldg_count', 'bldg_total_area', 'sales_total', 'float_total', 'work_total']
summary_cols = [c for c in summary_cols if c in df_analysis.columns]
display(df_analysis[summary_cols].describe().round(2))

=== 통합 데이터 요약 ===


,cong_avg,bldg_count,bldg_total_area,sales_total,float_total,work_total
count,119.00,119.00,119.00,116.00,116.00,115.00
mean,31.33,2010.88,161079.91,1145.02,668.09,26.16
std,10.92,1609.71,125319.63,1351.25,389.90,49.23
min,1.24,55.00,0.00,44.05,70.34,0.00
25%,26.97,831.00,68925.30,338.55,422.79,2.03
50%,32.72,1723.00,137491.78,589.74,580.21,5.85
75%,38.11,2540.50,229584.10,1298.81,768.61,18.73
max,51.48,7533.00,662355.62,5968.95,2090.06,250.90


## 4. 종합 상관관계 분석

In [14]:
# 혼잡도와 모든 변수 간 상관관계
cong_cols = ['cong_avg', 'cong_06_11', 'cong_11_14', 'cong_17_21', 'cong_21_24']
cong_cols = [c for c in cong_cols if c in df_analysis.columns]

feature_cols = [
    # 건물
    'bldg_count', 'bldg_avg_height', 'bldg_total_area', 'bldg_households',
    'bldg_주거', 'bldg_상업', 'bldg_업무',
    # 매출
    'sales_total', 'sales_06_11', 'sales_11_14', 'sales_17_21', 'sales_21_24',
    # 유동인구
    'float_total', 'float_06_11', 'float_17_21',
    # 직장인구
    'work_total', 'work_age20', 'work_age30'
]
feature_cols = [c for c in feature_cols if c in df_analysis.columns]

# 상관계수 계산
corr_matrix = df_analysis[cong_cols + feature_cols].corr()
corr_with_cong = corr_matrix.loc[feature_cols, cong_cols]

print("=== 혼잡도와의 상관계수 ===")
display(corr_with_cong.round(3))

=== 혼잡도와의 상관계수 ===


,cong_avg,cong_06_11,cong_11_14,cong_17_21,cong_21_24
bldg_count,0.151,0.148,0.186,0.122,0.147
bldg_avg_height,0.206,0.151,0.163,0.223,0.238
bldg_total_area,0.300,0.307,0.304,0.246,0.298
bldg_households,0.265,0.271,0.277,0.218,0.267
bldg_주거,0.111,0.186,0.109,0.054,0.070
bldg_상업,0.181,0.128,0.222,0.169,0.203
bldg_업무,0.361,0.240,0.358,0.376,0.419
sales_total,0.158,0.025,0.163,0.194,0.205
sales_06_11,0.189,0.058,0.192,0.230,0.220
sales_11_14,0.164,0.016,0.169,0.210,0.204


In [15]:
# 종합 상관관계 히트맵
fig = go.Figure(data=go.Heatmap(
    z=corr_with_cong.values,
    x=['평균혼잡도', '출근(06-11)', '점심(11-14)', '퇴근(17-21)', '야간(21-24)'],
    y=corr_with_cong.index,
    colorscale='RdBu_r',
    zmid=0,
    text=corr_with_cong.values.round(2),
    texttemplate='%{text}',
    textfont={'size': 9},
    colorbar=dict(title='상관계수')
))

fig.update_layout(
    title='시간대별 혼잡도와 도시 특성 종합 상관관계',
    height=600,
    width=700
)
fig.show()

In [16]:
# 평균 혼잡도와의 상관계수 순위
corr_avg = corr_with_cong['cong_avg'].sort_values(key=abs, ascending=False)

print("=== 평균 혼잡도와의 상관계수 (절대값 순) ===")
for idx, val in corr_avg.items():
    direction = "↑" if val > 0 else "↓"
    bar = "█" * int(abs(val) * 20)
    print(f"{idx:20s}: {val:+.3f} {direction} {bar}")

=== 평균 혼잡도와의 상관계수 (절대값 순) ===
bldg_업무             : +0.361 ↑ ███████
float_06_11         : +0.322 ↑ ██████
float_17_21         : +0.307 ↑ ██████
float_total         : +0.304 ↑ ██████
bldg_total_area     : +0.300 ↑ ██████
bldg_households     : +0.265 ↑ █████
sales_21_24         : +0.239 ↑ ████
work_age20          : +0.226 ↑ ████
work_total          : +0.220 ↑ ████
work_age30          : +0.207 ↑ ████
bldg_avg_height     : +0.206 ↑ ████
sales_06_11         : +0.189 ↑ ███
bldg_상업             : +0.181 ↑ ███
sales_11_14         : +0.164 ↑ ███
sales_17_21         : +0.159 ↑ ███
sales_total         : +0.158 ↑ ███
bldg_count          : +0.151 ↑ ███
bldg_주거             : +0.111 ↑ ██


## 5. 카테고리별 상세 분석

In [17]:
# 건물 특성과 혼잡도
print("=== 건물 특성과 혼잡도 상관관계 ===")
bldg_cols = [c for c in df_analysis.columns if c.startswith('bldg_') and not c.endswith('_ratio')]

fig = make_subplots(rows=2, cols=3, subplot_titles=[
    '건물 수', '평균 높이', '총 연면적', '주거 건물', '상업 건물', '업무 건물'
])

plot_cols = ['bldg_count', 'bldg_avg_height', 'bldg_total_area', 'bldg_주거', 'bldg_상업', 'bldg_업무']
positions = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

for col, pos in zip(plot_cols, positions):
    if col in df_analysis.columns:
        corr = df_analysis[col].corr(df_analysis['cong_avg'])
        fig.add_trace(
            go.Scatter(
                x=df_analysis[col], y=df_analysis['cong_avg'],
                mode='markers', marker=dict(size=5, opacity=0.6),
                text=df_analysis['station_name_kr'],
                name=f'r={corr:.2f}', showlegend=False
            ),
            row=pos[0], col=pos[1]
        )

fig.update_layout(height=500, title_text='건물 특성과 평균 혼잡도')
fig.show()

=== 건물 특성과 혼잡도 상관관계 ===


In [18]:
# 매출과 혼잡도
print("=== 매출과 혼잡도 상관관계 ===")

if 'sales_total' in df_analysis.columns:
    fig = make_subplots(rows=2, cols=3, subplot_titles=[
        '총 매출', '출근대 매출', '점심 매출', '오후 매출', '퇴근대 매출', '야간 매출'
    ])

    sales_cols = ['sales_total', 'sales_06_11', 'sales_11_14', 'sales_14_17', 'sales_17_21', 'sales_21_24']
    positions = [(1,1), (1,2), (1,3), (2,1), (2,2), (2,3)]

    for col, pos in zip(sales_cols, positions):
        if col in df_analysis.columns:
            corr = df_analysis[col].corr(df_analysis['cong_avg'])
            fig.add_trace(
                go.Scatter(
                    x=df_analysis[col], y=df_analysis['cong_avg'],
                    mode='markers', marker=dict(size=5, opacity=0.6, color='orange'),
                    text=df_analysis['station_name_kr'],
                    showlegend=False
                ),
                row=pos[0], col=pos[1]
            )

    fig.update_layout(height=500, title_text='매출과 평균 혼잡도')
    fig.update_xaxes(title_text='매출(억)')
    fig.show()

=== 매출과 혼잡도 상관관계 ===


In [19]:
# 유동인구/직장인구와 혼잡도
print("=== 인구와 혼잡도 상관관계 ===")

fig = make_subplots(rows=1, cols=3, subplot_titles=['유동인구', '직장인구', '유동+직장'])

if 'float_total' in df_analysis.columns:
    corr = df_analysis['float_total'].corr(df_analysis['cong_avg'])
    fig.add_trace(
        go.Scatter(x=df_analysis['float_total'], y=df_analysis['cong_avg'],
                   mode='markers', marker=dict(size=6, opacity=0.6, color='green'),
                   text=df_analysis['station_name_kr'], name=f'r={corr:.2f}'),
        row=1, col=1
    )

if 'work_total' in df_analysis.columns:
    corr = df_analysis['work_total'].corr(df_analysis['cong_avg'])
    fig.add_trace(
        go.Scatter(x=df_analysis['work_total'], y=df_analysis['cong_avg'],
                   mode='markers', marker=dict(size=6, opacity=0.6, color='purple'),
                   text=df_analysis['station_name_kr'], name=f'r={corr:.2f}'),
        row=1, col=2
    )

if 'float_total' in df_analysis.columns and 'work_total' in df_analysis.columns:
    df_analysis['pop_combined'] = df_analysis['float_total'].fillna(0) + df_analysis['work_total'].fillna(0)
    corr = df_analysis['pop_combined'].corr(df_analysis['cong_avg'])
    fig.add_trace(
        go.Scatter(x=df_analysis['pop_combined'], y=df_analysis['cong_avg'],
                   mode='markers', marker=dict(size=6, opacity=0.6, color='red'),
                   text=df_analysis['station_name_kr'], name=f'r={corr:.2f}'),
        row=1, col=3
    )

fig.update_layout(height=400, title_text='인구와 평균 혼잡도')
fig.update_xaxes(title_text='인구(만명)')
fig.update_yaxes(title_text='혼잡도')
fig.show()

=== 인구와 혼잡도 상관관계 ===


## 6. 시간대별 패턴 분석

In [20]:
# 시간대별 매출-혼잡도 상관계수 (동일 시간대)
time_pairs = [
    ('sales_06_11', 'cong_06_11', '06-11시'),
    ('sales_11_14', 'cong_11_14', '11-14시'),
    ('sales_14_17', 'cong_14_17', '14-17시'),
    ('sales_17_21', 'cong_17_21', '17-21시'),
    ('sales_21_24', 'cong_21_24', '21-24시')
]

print("=== 동일 시간대 매출-혼잡도 상관계수 ===")
same_time_corrs = []
for sales_col, cong_col, label in time_pairs:
    if sales_col in df_analysis.columns and cong_col in df_analysis.columns:
        corr = df_analysis[sales_col].corr(df_analysis[cong_col])
        same_time_corrs.append({'시간대': label, '상관계수': corr})
        print(f"{label}: r = {corr:.3f}")

df_time_corr = pd.DataFrame(same_time_corrs)

=== 동일 시간대 매출-혼잡도 상관계수 ===
06-11시: r = 0.058
11-14시: r = 0.169
14-17시: r = 0.116
17-21시: r = 0.192
21-24시: r = 0.299


In [21]:
# 시간대별 상관계수 시각화
if len(df_time_corr) > 0:
    fig = go.Figure(data=go.Bar(
        x=df_time_corr['시간대'],
        y=df_time_corr['상관계수'],
        marker_color=['blue' if x > 0 else 'red' for x in df_time_corr['상관계수']],
        text=df_time_corr['상관계수'].round(3),
        textposition='outside'
    ))

    fig.update_layout(
        title='시간대별 매출-혼잡도 상관계수',
        xaxis_title='시간대',
        yaxis_title='상관계수',
        height=400
    )
    fig.add_hline(y=0, line_dash='dash', line_color='gray')
    fig.show()

## 7. 다중 회귀 분석 (혼잡도 예측)

In [22]:
# 회귀 분석용 피처 선택
reg_features = [
    'bldg_count', 'bldg_avg_height', 'bldg_total_area',
    'bldg_주거', 'bldg_상업', 'bldg_업무',
    'sales_total', 'float_total', 'work_total'
]
reg_features = [f for f in reg_features if f in df_analysis.columns]

# 결측치 처리
df_reg = df_analysis[['station_name_kr', 'cong_avg'] + reg_features].dropna()

print(f"회귀 분석 대상: {len(df_reg)} 역")
print(f"사용 피처: {reg_features}")

회귀 분석 대상: 115 역
사용 피처: ['bldg_count', 'bldg_avg_height', 'bldg_total_area', 'bldg_주거', 'bldg_상업', 'bldg_업무', 'sales_total', 'float_total', 'work_total']


In [23]:
# 선형 회귀 분석
X = df_reg[reg_features].fillna(0)
y = df_reg['cong_avg']

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 선형 회귀
lr = LinearRegression()
lr.fit(X_scaled, y)

# 계수 출력
coef_df = pd.DataFrame({
    '변수': reg_features,
    '계수': lr.coef_,
    '절대값': np.abs(lr.coef_)
}).sort_values('절대값', ascending=False)

print("=== 선형 회귀 계수 (표준화) ===")
print(f"R² Score: {lr.score(X_scaled, y):.3f}")
print()
for _, row in coef_df.iterrows():
    direction = "+" if row['계수'] > 0 else "-"
    bar = "█" * int(row['절대값'] * 5)
    print(f"{row['변수']:20s}: {row['계수']:+.3f} {direction} {bar}")

=== 선형 회귀 계수 (표준화) ===
R² Score: 0.257

bldg_total_area     : +3.879 + ███████████████████
bldg_count          : +3.667 + ██████████████████
bldg_주거             : -3.610 - ██████████████████
bldg_상업             : -2.122 - ██████████
bldg_업무             : +1.955 + █████████
work_total          : +1.851 + █████████
float_total         : +1.794 + ████████
sales_total         : -1.505 - ███████
bldg_avg_height     : +0.817 + ████


In [24]:
# 랜덤 포레스트로 변수 중요도
rf = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)
rf.fit(X, y)

# 변수 중요도
importance_df = pd.DataFrame({
    '변수': reg_features,
    '중요도': rf.feature_importances_
}).sort_values('중요도', ascending=False)

print("=== 랜덤 포레스트 변수 중요도 ===")
print(f"R² Score: {rf.score(X, y):.3f}")
print()

fig = go.Figure(data=go.Bar(
    x=importance_df['변수'],
    y=importance_df['중요도'],
    marker_color='steelblue',
    text=importance_df['중요도'].round(3),
    textposition='outside'
))

fig.update_layout(
    title='혼잡도 예측 변수 중요도 (Random Forest)',
    xaxis_title='변수',
    yaxis_title='중요도',
    height=400,
    xaxis_tickangle=-45
)
fig.show()

=== 랜덤 포레스트 변수 중요도 ===
R² Score: 0.775



In [25]:
# 예측값 vs 실제값
df_reg['predicted'] = rf.predict(X)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_reg['cong_avg'],
    y=df_reg['predicted'],
    mode='markers',
    marker=dict(size=8, opacity=0.6),
    text=df_reg['station_name_kr'],
    hovertemplate='%{text}<br>실제: %{x:.1f}<br>예측: %{y:.1f}<extra></extra>'
))

# 대각선
max_val = max(df_reg['cong_avg'].max(), df_reg['predicted'].max())
fig.add_trace(go.Scatter(
    x=[0, max_val], y=[0, max_val],
    mode='lines', line=dict(color='red', dash='dash'),
    name='완벽한 예측'
))

fig.update_layout(
    title=f'혼잡도 예측 결과 (R² = {rf.score(X, y):.3f})',
    xaxis_title='실제 혼잡도',
    yaxis_title='예측 혼잡도',
    height=500
)
fig.show()

## 8. 역 유형 분류 (클러스터링)

In [26]:
from sklearn.cluster import KMeans

# 클러스터링 피처
cluster_features = ['bldg_주거_ratio', 'bldg_상업_ratio', 'bldg_업무_ratio', 'cong_avg']
cluster_features = [f for f in cluster_features if f in df_analysis.columns]

if len(cluster_features) >= 3:
    df_cluster = df_analysis.dropna(subset=cluster_features).copy()
    X_cluster = df_cluster[cluster_features].fillna(0)
    
    # 스케일링
    X_cluster_scaled = StandardScaler().fit_transform(X_cluster)
    
    # K-means
    kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
    df_cluster['cluster'] = kmeans.fit_predict(X_cluster_scaled)
    
    print("=== 클러스터별 역 수 ===")
    print(df_cluster['cluster'].value_counts().sort_index())
    
    # 클러스터별 특성
    cluster_profile = df_cluster.groupby('cluster')[cluster_features + ['cong_06_11', 'cong_17_21']].mean()
    print("\n=== 클러스터별 특성 ===")
    display(cluster_profile.round(3))

=== 클러스터별 역 수 ===
cluster
0    25
1    53
2    30
3    11
Name: count, dtype: int64

=== 클러스터별 특성 ===


,bldg_주거_ratio,bldg_상업_ratio,bldg_업무_ratio,cong_avg,cong_06_11,cong_17_21
cluster,,,,,,
0,0.434,0.148,0.012,15.226,20.928,21.408
1,0.554,0.163,0.016,35.362,44.240,49.966
2,0.311,0.356,0.030,34.211,40.128,48.815
3,0.171,0.306,0.176,40.659,43.307,62.049


In [27]:
# 클러스터 시각화
if 'cluster' in df_cluster.columns:
    fig = px.scatter(
        df_cluster,
        x='bldg_주거_ratio' if 'bldg_주거_ratio' in df_cluster.columns else cluster_features[0],
        y='cong_avg',
        color='cluster',
        hover_name='station_name_kr',
        title='역 유형 클러스터링',
        labels={'cluster': '클러스터'},
        color_continuous_scale='Viridis'
    )
    fig.update_layout(height=500)
    fig.show()
    
    # 클러스터별 대표 역
    print("\n=== 클러스터별 대표 역 ===")
    for c in sorted(df_cluster['cluster'].unique()):
        stations = df_cluster[df_cluster['cluster'] == c].nlargest(5, 'cong_avg')['station_name_kr'].tolist()
        print(f"클러스터 {c}: {', '.join(stations)}")


=== 클러스터별 대표 역 ===
클러스터 0: 잠실나루, 동작, 이촌, 도림천, 숙대입구
클러스터 1: 방배, 낙성대, 길동, 남태령, 서울대입구
클러스터 2: 동대문, 혜화, 문래, 신당, 당산
클러스터 3: 서초, 교대, 강남, 역삼, 선릉


## 9. 종합 결과 요약

In [28]:
print("=" * 80)
print("시간대별 지하철 혼잡도 종합 연관성 분석 결과")
print("=" * 80)

print(f"""
[분석 개요]
  - 분석 대상: 2호선, 4호선, 5호선 ({len(df_analysis)}개 역)
  - 분석 데이터: 건물, 매출, 유동인구, 직장인구
""")

print("[평균 혼잡도와 주요 상관관계]")
top_corrs = corr_with_cong['cong_avg'].sort_values(key=abs, ascending=False).head(10)
for var, corr in top_corrs.items():
    print(f"  {var:20s}: r = {corr:+.3f}")

print(f"\n[회귀 분석 결과]")
print(f"  - 선형 회귀 R²: {lr.score(X_scaled, y):.3f}")
print(f"  - 랜덤 포레스트 R²: {rf.score(X, y):.3f}")
print(f"  - 가장 중요한 변수: {importance_df.iloc[0]['변수']}")

print(f"\n[주요 발견]")
print("  1. 건물 특성")
if 'bldg_업무' in corr_with_cong.index:
    print(f"     - 업무 건물 수와 혼잡도: r = {corr_with_cong.loc['bldg_업무', 'cong_avg']:.3f}")
if 'bldg_주거' in corr_with_cong.index:
    print(f"     - 주거 건물 수와 출근 혼잡도: r = {corr_with_cong.loc['bldg_주거', 'cong_06_11']:.3f}")

print("  2. 매출")
if 'sales_total' in corr_with_cong.index:
    print(f"     - 총 매출과 혼잡도: r = {corr_with_cong.loc['sales_total', 'cong_avg']:.3f}")
    
print("  3. 인구")
if 'work_total' in corr_with_cong.index:
    print(f"     - 직장인구와 혼잡도: r = {corr_with_cong.loc['work_total', 'cong_avg']:.3f}")

print(f"\n[시사점]")
print("  - 업무 건물이 많은 역: 전반적으로 높은 혼잡도")
print("  - 주거 건물이 많은 역: 출근 시간대 혼잡도 높음")
print("  - 매출과 혼잡도: 야간/퇴근 시간대에 강한 상관")
print("  - 직장인구가 많은 행정동: 전반적으로 높은 혼잡도")

conn.close()
print("\n" + "=" * 80)
print("분석 완료!")

시간대별 지하철 혼잡도 종합 연관성 분석 결과

[분석 개요]
  - 분석 대상: 2호선, 4호선, 5호선 (119개 역)
  - 분석 데이터: 건물, 매출, 유동인구, 직장인구

[평균 혼잡도와 주요 상관관계]
  bldg_업무             : r = +0.361
  float_06_11         : r = +0.322
  float_17_21         : r = +0.307
  float_total         : r = +0.304
  bldg_total_area     : r = +0.300
  bldg_households     : r = +0.265
  sales_21_24         : r = +0.239
  work_age20          : r = +0.226
  work_total          : r = +0.220
  work_age30          : r = +0.207

[회귀 분석 결과]
  - 선형 회귀 R²: 0.257
  - 랜덤 포레스트 R²: 0.775
  - 가장 중요한 변수: bldg_업무

[주요 발견]
  1. 건물 특성
     - 업무 건물 수와 혼잡도: r = 0.361
     - 주거 건물 수와 출근 혼잡도: r = 0.186
  2. 매출
     - 총 매출과 혼잡도: r = 0.158
  3. 인구
     - 직장인구와 혼잡도: r = 0.220

[시사점]
  - 업무 건물이 많은 역: 전반적으로 높은 혼잡도
  - 주거 건물이 많은 역: 출근 시간대 혼잡도 높음
  - 매출과 혼잡도: 야간/퇴근 시간대에 강한 상관
  - 직장인구가 많은 행정동: 전반적으로 높은 혼잡도

분석 완료!
